In [9]:
import findspark
findspark.init()
# Spark:
from pyspark     import SparkConf
from pyspark     import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
# Python :
from datetime import datetime
import math
from pyspark.streaming import StreamingContext

listen_to_ip   = "54.37.12.49"
listen_to_port = 12012

current_date   = datetime.now().strftime("%Y-%m-%d %H:%M")
user           = "romain - exemple saprk streaming"
appName        = "%s le %s"%(user, current_date)
ip_fares       = "54.37.12.49"
master         = "spark://54.37.12.49:7077"
executor_cores = 2 
nb_cores_max   = 10
parallelism    = 4 * executor_cores
memory_by_node = "200M"
nb_executor    = nb_cores_max//executor_cores
dico_conf      = { "spark.app.name"             : appName        ,
                   "spark.master"               : master         ,
                   "spark.cores.max"            : nb_cores_max   , 
                   "spark.executor.cores"       : executor_cores ,
                   "spark.default.parallelism"  : parallelism    , 
                  #"spark.python.worker.memory" : memory_by_node ,
                   "spark.python.executor.memory" : memory_by_node }

conf = SparkConf()
for k,v in dico_conf.items():
    conf = conf.set(k,v)
sc = SparkContext.getOrCreate(conf=conf)

nb_secondes = 1
ssc         = StreamingContext(sc, nb_secondes)
dstream     = ssc.socketTextStream(listen_to_ip, listen_to_port)

In [10]:
ssc.checkpoint("hdfs://54.37.12.49:8020/spark_streaming_checkpoint/")

In [11]:
def update_achats(nouvelles_valeurs, valeur_actuelle ):
    if valeur_actuelle is None:
        valeur_actuelle = 0
    return sum(nouvelles_valeurs, valeur_actuelle)
        

In [12]:
data            = dstream.map(lambda x: x.split(","))
clients_facture = data.map(lambda x: (x[0], float(x[1])*float(x[2])))
update_client   = clients_facture.updateStateByKey(update_achats)
update_client.pprint()

# 9) activer l'écoute sur la socket

In [13]:
ssc.start()

-------------------------------------------
Time: 2018-02-26 14:32:05
-------------------------------------------

-------------------------------------------
Time: 2018-02-26 14:32:06
-------------------------------------------
('client_b', 160.0)
('client_a', 13.0)
('client_d', 480.0)

-------------------------------------------
Time: 2018-02-26 14:32:07
-------------------------------------------
('client_b', 516.0)
('client_a', 683.0)
('client_d', 814.0)

-------------------------------------------
Time: 2018-02-26 14:32:08
-------------------------------------------
('client_c', 119.0)
('client_b', 560.0)
('client_a', 819.0)
('client_d', 1091.0)

-------------------------------------------
Time: 2018-02-26 14:32:09
-------------------------------------------
('client_c', 323.0)
('client_b', 738.0)
('client_a', 1719.0)
('client_d', 1226.0)

-------------------------------------------
Time: 2018-02-26 14:32:10
-------------------------------------------
('client_c', 620.0)
('client_

In [14]:
ssc.stop(False, True)

-------------------------------------------
Time: 2018-02-26 14:32:45
-------------------------------------------
('client_c', 1682.0)
('client_b', 3067.0)
('client_a', 4450.0)
('client_d', 3639.0)

-------------------------------------------
Time: 2018-02-26 14:32:46
-------------------------------------------
('client_c', 1682.0)
('client_b', 3067.0)
('client_a', 4450.0)
('client_d', 3639.0)

-------------------------------------------
Time: 2018-02-26 14:32:47
-------------------------------------------
('client_c', 1682.0)
('client_b', 3067.0)
('client_a', 4450.0)
('client_d', 3639.0)

-------------------------------------------
Time: 2018-02-26 14:32:48
-------------------------------------------
('client_c', 1682.0)
('client_b', 3067.0)
('client_a', 4450.0)
('client_d', 3639.0)

-------------------------------------------
Time: 2018-02-26 14:32:49
-------------------------------------------
('client_c', 1682.0)
('client_b', 3067.0)
('client_a', 4450.0)
('client_d', 3639.0)

-----